In [1]:
from cloudvolume import CloudVolume
from cloudvolume.exceptions import SkeletonDecodeError
from itertools import islice
from skimage.transform import downscale_local_mean
import napari
from skimage import io
import random
import h5py
from skimage import measure
from brainlit.preprocessing import removeSmallCCs
import numpy as np 
import matplotlib.pyplot as plt 
import subprocess
import tables
from napari_animation import AnimationWidget
from tqdm import tqdm
import pickle
import networkx as nx
import seaborn as sns
import pandas as pd
import brainrender
from brainlit.utils.session import NeuroglancerSession
from skimage.filters import threshold_otsu, threshold_local
from brainlit.preprocessing import split_frags, rename_states_consecutively, label_points
from brainlit.algorithms.connect_fragments import most_probable_neuron_path
from brainlit.algorithms.connect_fragments import trace_evaluation
import similaritymeasures
from brainlit.viz import Bresenham3D
from cloudvolume import Skeleton
from sklearn.metrics import pairwise_distances_argmin_min
import time
%gui qt5

/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


embedWindow(verbose=True): could not load ipyvtk_simple try:
> pip install -U git+https://github.com/Kitware/ipyvtk-simple.git


In [20]:
dir = "s3://open-neurodata/brainlit/brain1"
dir_segments = "s3://open-neurodata/brainlit/brain1_segments"

ngl_sess = NeuroglancerSession(mip = 0, url = dir, url_segments=dir_segments)

res = [0.3,0.3,1]
threshold = 0.9

block_size = 101

img, bbox, vox = ngl_sess.pull_vertex_list(2, [0], 250)
print("{:e}".format(np.product(img.shape)))

Downloading:  99%|█████████▉| 297/300 [00:08<00:00, 42.35it/s]

1.257515e+08


Downloading: 100%|██████████| 300/300 [00:18<00:00, 42.35it/s]

In [22]:
#view
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(img)
viewer.add_points(vox, size=7)
napari.run()

In [23]:
io.imsave("/Users/thomasathey/Documents/mimlab/mouselight/input/images/gui/250.tif", img)

In [27]:
pred_path ="/Users/thomasathey/Documents/mimlab/mouselight/input/images/gui/250_Probabilities.h5"
f = h5py.File(pred_path, 'r')
pred = f.get('exported_data')
pred = pred[:,:,:,1]
im_processed = pred

mask = im_processed >= threshold
labels = measure.label(mask)

soma_coords = vox[0:1,:]

In [28]:
new_labels = split_frags(soma_coords, labels, im_processed, threshold, res)
_, soma_lbls = label_points(new_labels, soma_coords, res)
soma_lbl = soma_lbls[0]

In [30]:
viterbi = most_probable_neuron_path(image=img.astype(float), labels=new_labels, soma_lbls=[soma_lbl], resolution=res, coef_dist=10, coef_curv=1000)
viterbi.frags_to_lines()
viterbi.reset_dists(type="all")
viterbi.compute_all_costs_dist(point_point_func=viterbi.point_point_dist, point_blob_func=viterbi.point_blob_dist)
viterbi.compute_all_costs_int()
viterbi.create_nx_graph()

Setting up emission distribution...


Adding edges to nx graph: 100%|██████████| 4757/4757 [01:18<00:00, 60.87it/s]


In [31]:
with open('/Users/thomasathey/Documents/mimlab/mouselight/input/images/gui/viterbi_250.pickle', 'wb') as handle:
    pickle.dump(viterbi, handle)